# **bytebuddy**

## unicode code points

In [1]:
s = 'hello, world'
t = [ord(c) for c in s]
s = [chr(i) for i in t]
s, t

(['h', 'e', 'l', 'l', 'o', ',', ' ', 'w', 'o', 'r', 'l', 'd'],
 [104, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100])

## unicode byte encodings

In [2]:
s = 'こんにちは世界'
print(s.encode('utf-8'))
print(s.encode('utf-16'))
print(s.encode('utf-32'))

b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf\xe4\xb8\x96\xe7\x95\x8c'
b'\xff\xfeS0\x930k0a0o0\x16NLu'
b'\xff\xfe\x00\x00S0\x00\x00\x930\x00\x00k0\x00\x00a0\x00\x00o0\x00\x00\x16N\x00\x00Lu\x00\x00'


In [3]:
print(list(s.encode('utf-8')))
print(list(s.encode('utf-16')))
print(list(s.encode('utf-32')))

[227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 228, 184, 150, 231, 149, 140]
[255, 254, 83, 48, 147, 48, 107, 48, 97, 48, 111, 48, 22, 78, 76, 117]
[255, 254, 0, 0, 83, 48, 0, 0, 147, 48, 0, 0, 107, 48, 0, 0, 97, 48, 0, 0, 111, 48, 0, 0, 22, 78, 0, 0, 76, 117, 0, 0]


In [4]:
text = 'Hello, 世界! 🌍😊 Привет мир! 🚀🌟 नमस्ते दुनिया! 🎉📚 こんにちは世界! 🎌🐉 안녕하세요 세계! 🎵🖋️'
tokens = text.encode('utf-8')   # raw bytes
tokens = list(tokens)           # convert to a list of integers in range 0...255 for convenience
print('----------')
print(text)
print('length:', len(text))
print('----------')
print(tokens)
print('length:', len(tokens))

----------
Hello, 世界! 🌍😊 Привет мир! 🚀🌟 नमस्ते दुनिया! 🎉📚 こんにちは世界! 🎌🐉 안녕하세요 세계! 🎵🖋️
length: 72
----------
[72, 101, 108, 108, 111, 44, 32, 228, 184, 150, 231, 149, 140, 33, 32, 240, 159, 140, 141, 240, 159, 152, 138, 32, 208, 159, 209, 128, 208, 184, 208, 178, 208, 181, 209, 130, 32, 208, 188, 208, 184, 209, 128, 33, 32, 240, 159, 154, 128, 240, 159, 140, 159, 32, 224, 164, 168, 224, 164, 174, 224, 164, 184, 224, 165, 141, 224, 164, 164, 224, 165, 135, 32, 224, 164, 166, 224, 165, 129, 224, 164, 168, 224, 164, 191, 224, 164, 175, 224, 164, 190, 33, 32, 240, 159, 142, 137, 240, 159, 147, 154, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 228, 184, 150, 231, 149, 140, 33, 32, 240, 159, 142, 140, 240, 159, 144, 137, 32, 236, 149, 136, 235, 133, 149, 237, 149, 152, 236, 132, 184, 236, 154, 148, 32, 236, 132, 184, 234, 179, 132, 33, 32, 240, 159, 142, 181, 240, 159, 150, 139, 239, 184, 143]
length: 169


In [5]:
text = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
tokens = text.encode('utf-8')   # raw bytes
tokens = list(tokens)           # convert to a list of integers in range 0...255 for convenience
print('----------')
print(text)
print('length:', len(text))
print('----------')
print(tokens)
print('length:', len(tokens))

----------
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
length: 445
----------
[76, 111, 114, 101, 109, 32, 105, 112, 115, 117, 109, 32, 100, 111, 108, 111, 114, 32, 115, 105, 116, 32, 97, 109, 101, 116, 44, 32, 99, 111, 110, 115, 101, 99, 116, 101, 116, 117, 114, 32, 97, 100, 105, 112, 105, 115, 99, 105, 110, 103, 32, 101, 108, 105, 116, 44, 32, 115, 101, 100, 32, 100, 111, 32, 101, 105, 117, 115, 109, 111, 100, 32, 116, 101, 109, 112, 111, 114, 32, 105, 110, 99, 105, 100, 105, 100, 117, 110, 116, 32, 117, 116, 32, 108, 97, 98, 111, 114, 101, 32, 101, 116, 32, 100, 111, 108, 111, 114, 101, 3

## **byte-pair encoding (BPE) algorithm**

In [6]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):      # pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [7]:
stats = get_stats(tokens)
print(stats)
print(sorted(((v, k) for k, v in stats.items()), reverse=True))

{(76, 111): 1, (111, 114): 9, (114, 101): 7, (101, 109): 2, (109, 32): 6, (32, 105): 7, (105, 112): 3, (112, 115): 1, (115, 117): 2, (117, 109): 3, (32, 100): 6, (100, 111): 6, (111, 108): 6, (108, 111): 4, (114, 32): 5, (32, 115): 4, (115, 105): 3, (105, 116): 6, (116, 32): 16, (32, 97): 7, (97, 109): 3, (109, 101): 1, (101, 116): 3, (116, 44): 3, (44, 32): 4, (32, 99): 6, (99, 111): 4, (111, 110): 4, (110, 115): 2, (115, 101): 5, (101, 99): 2, (99, 116): 1, (116, 101): 5, (116, 117): 2, (117, 114): 4, (97, 100): 2, (100, 105): 2, (112, 105): 2, (105, 115): 5, (115, 99): 1, (99, 105): 5, (105, 110): 7, (110, 103): 1, (103, 32): 1, (32, 101): 10, (101, 108): 2, (108, 105): 5, (101, 100): 1, (100, 32): 5, (111, 32): 3, (101, 105): 1, (105, 117): 1, (117, 115): 1, (115, 109): 1, (109, 111): 3, (111, 100): 2, (32, 116): 1, (109, 112): 1, (112, 111): 1, (110, 99): 1, (105, 100): 5, (100, 117): 1, (117, 110): 3, (110, 116): 5, (32, 117): 3, (117, 116): 3, (32, 108): 3, (108, 97): 5, (97, 98

In [8]:
top_pair = max(stats, key=stats.get)
top_pair

(116, 32)

In [9]:
def merge(ids, pair, idx):
    # in the list of int(s) (ids), replace all consecutive occurences of pair with the new token idx
    newIds = []

    i = 0
    while i < len(ids):
        # if we are not at the very last position AND the pair matches, replace it
        if i < len(ids)-1 and pair[0] == ids[i] and pair[1] == ids[i+1]:
            newIds.append(idx)
            i += 2
        else:
            newIds.append(ids[i])
            i += 1
            
    return newIds

In [10]:
tokens = merge(tokens, top_pair, 256)
print(tokens)
print('length:', len(tokens))

[76, 111, 114, 101, 109, 32, 105, 112, 115, 117, 109, 32, 100, 111, 108, 111, 114, 32, 115, 105, 256, 97, 109, 101, 116, 44, 32, 99, 111, 110, 115, 101, 99, 116, 101, 116, 117, 114, 32, 97, 100, 105, 112, 105, 115, 99, 105, 110, 103, 32, 101, 108, 105, 116, 44, 32, 115, 101, 100, 32, 100, 111, 32, 101, 105, 117, 115, 109, 111, 100, 32, 116, 101, 109, 112, 111, 114, 32, 105, 110, 99, 105, 100, 105, 100, 117, 110, 256, 117, 256, 108, 97, 98, 111, 114, 101, 32, 101, 256, 100, 111, 108, 111, 114, 101, 32, 109, 97, 103, 110, 97, 32, 97, 108, 105, 113, 117, 97, 46, 32, 85, 256, 101, 110, 105, 109, 32, 97, 100, 32, 109, 105, 110, 105, 109, 32, 118, 101, 110, 105, 97, 109, 44, 32, 113, 117, 105, 115, 32, 110, 111, 115, 116, 114, 117, 100, 32, 101, 120, 101, 114, 99, 105, 116, 97, 116, 105, 111, 110, 32, 117, 108, 108, 97, 109, 99, 111, 32, 108, 97, 98, 111, 114, 105, 115, 32, 110, 105, 115, 105, 32, 117, 256, 97, 108, 105, 113, 117, 105, 112, 32, 101, 120, 32, 101, 97, 32, 99, 111, 109, 109, 1

In [11]:
text = open('tiny_shakespeare.txt', 'r', encoding='utf-8').read()
tokens = list(text.encode('utf-8'))
print(tokens[:80])
print(len(tokens))

[70, 105, 114, 115, 116, 32, 67, 105, 116, 105, 122, 101, 110, 58, 10, 66, 101, 102, 111, 114, 101, 32, 119, 101, 32, 112, 114, 111, 99, 101, 101, 100, 32, 97, 110, 121, 32, 102, 117, 114, 116, 104, 101, 114, 44, 32, 104, 101, 97, 114, 32, 109, 101, 32, 115, 112, 101, 97, 107, 46, 10, 10, 65, 108, 108, 58, 10, 83, 112, 101, 97, 107, 44, 32, 115, 112, 101, 97, 107, 46]
1115394


### training the tokenizer

In [12]:
vocab_size = 324            # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens)          # copy so we don't destroy the original list


merges = {}                 # (int, int) -> (int)
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)

    idx = 256 + i
    ids = merge(ids, pair, idx)

    print(f'merging {pair} into a new token {idx}')
    merges[pair] = idx

merging (101, 32) into a new token 256
merging (116, 104) into a new token 257
merging (116, 32) into a new token 258
merging (115, 32) into a new token 259
merging (100, 32) into a new token 260
merging (44, 32) into a new token 261
merging (111, 117) into a new token 262
merging (101, 114) into a new token 263
merging (105, 110) into a new token 264
merging (121, 32) into a new token 265
merging (97, 110) into a new token 266
merging (58, 10) into a new token 267
merging (111, 114) into a new token 268
merging (111, 32) into a new token 269
merging (101, 110) into a new token 270
merging (10, 10) into a new token 271
merging (97, 114) into a new token 272
merging (32, 257) into a new token 273
merging (111, 110) into a new token 274
merging (108, 108) into a new token 275
merging (104, 97) into a new token 276
merging (44, 10) into a new token 277
merging (46, 271) into a new token 278
merging (105, 259) into a new token 279
merging (101, 115) into a new token 280
merging (121, 262) 

In [13]:
vocab = {idx : bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

In [14]:
print('token length:', len(tokens))
print('ids length:', len(ids))
print(f'compression ratio: {len(tokens)/len(ids):.2f}x')

token length: 1115394
ids length: 736256
compression ratio: 1.51x


**Note:** The tokenizer is a completely seperate, independent module from the LLM. It has its own training dataset of text (which could be different from that of the LLM), on which you train the vocabulary using the Byte Pair Encoding (BPE) algorithm. It then translates back and forth between raw text and sequences of tokens. The LLM later only ever sees the tokens and never directly deals with any text.

<img src="images/tokenizer_llm.png"/>

## **decoding**

In [15]:
def decode(ids):
    # given ids (list of integers), return a Python string
    tokens = b''.join(vocab[idx] for idx in ids)
    text = tokens.decode('utf-8', errors='replace')
    return text

In [16]:
print(decode(ids)[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


## **encoding**

In [17]:
def encode(text):
    # given a string, return a list of integers (the tokens)
    tokens = list(text.encode('utf-8'))

    while len(tokens) > 1:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float('inf')))
        
        # nothing else can be merged
        if pair not in merges:
            break   
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)                

    return tokens

In [18]:
print(encode(text)[:80])

[70, 299, 310, 67, 316, 105, 122, 270, 267, 66, 101, 300, 256, 119, 256, 112, 114, 111, 99, 101, 101, 260, 266, 265, 102, 117, 114, 257, 263, 261, 104, 314, 287, 256, 115, 112, 286, 107, 278, 65, 275, 267, 83, 112, 286, 107, 261, 115, 112, 286, 107, 278, 70, 299, 310, 67, 316, 105, 122, 270, 267, 89, 262, 32, 272, 256, 97, 308, 114, 280, 111, 108, 118, 101, 260, 114, 97, 257, 322, 283]


In [19]:
val_text = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
print(val_text == decode(encode(val_text)))

True
